In [ ]:
import pandas as pd
import bs4 as bs 
#import BeautifulSoup as bs
%pylab inline
import requests
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
import requests
import random

headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 92.0 Win10
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 91.0 Win10
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Firefox 90.0 Win10
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]

In [ ]:
city_list = city_list = ['dallas', "houston", "chicago", "phoenix", "newyork", "philadelphia", "sandiego","losangeles", "sanantonio", "sfbay"]
links = []
for city in city_list:
    for i in range(1,6):
        url = f"https://{city}.craigslist.org/search/ppa#search=1~gallery~"+str(i)+"~0"
        browser = webdriver.Chrome('chromedriver')
        options = webdriver.ChromeOptions()
        options.headless = True
        options.add_argument('--enable-javascript')
        options.add_argument("--headless")
        browser.get(url)
        time.sleep(2)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(2)
        #bsobj = soup(page.content,'lxml')
        for link in soup.findAll(class_='titlestring'):
            links.append(link.get('href'))
        
      

In [ ]:
scraped_data = pd.DataFrame({'link':links})
scraped_data.shape

In [ ]:
scraped_data=scraped_data[['link']]

In [ ]:
import numpy as np
def get_listing_info(args):
    time.sleep(np.random.uniform(1))
    link = args[0]
    listing_heading = 'NA'
    listing_price = 'NA'
    condition_val = 'NA'
    description = 'NA'
    make_val='NA'
    headers=random.choice(headers_list)
    r=requests.Session()
    r.headers=headers
    page = r.get(link)
    bsobj = BeautifulSoup(page.content,'lxml')
    for link in bsobj.findAll('h1',{'class':'postingtitle'}):
        listing_heading = link.text
        heading_split = listing_heading.split()
        for tag in heading_split:
        if tag.startswith('$') and tag[-1].isdigit():
            listing_price = tag
    for link in bsobj.findAll('p',{'class':'attrgroup'}):
        temp_tag = link.text.split('\n')
        temp_tag = [val.strip() for val in temp_tag if val.startswith('condition')]
        if len(temp_tag) > 0:
        condition_val = temp_tag[0].split(':')[1].strip()
    for i in bsobj.findAll('section',{'id':'postingbody'}):
        description = i.text.strip().replace('QR Code Link to This Post','').replace('\n','')
    for link in bsobj.findAll('p',{'class':'attrgroup'}):
        make_tag=link.text.split('\n')
        make_tag=[val.strip() for val in make_tag if val.startswith('make')]
        if len(make_tag) > 0:
            make_val = make_tag[0].split(':')[1].strip()
    return pd.Series([listing_heading.strip(), description, listing_price, condition_val, make_val])

In [ ]:
scraped_data[['Title', 'Description' ,'Price', 'Condition','Make']] = scraped_data.apply(get_listing_info, axis=1,  result_type='expand')
scraped_data['Title'] = scraped_data['Title'].str.strip()




In [ ]:
finaldf.to_csv('Appliances_Postings_for_AUD.csv')